# Setup
Setting up connections to AI Studio

In [1]:
import json

# Import required libraries
from promptflow.azure import PFClient

# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from evaluate import run_azure_flow, run_azure_eval_flow

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [3]:
config_path = "../config.json"
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

Found the config file in: ../config.json


# Testing Contoso Chat w/ Sales Data
Running base flow

In [4]:
# Update the runtime to the name of the runtime you created previously
runtime = "automatic"
flow = "../contoso-chat"
data = "../data/salestestdata.jsonl"
run_name = "contoso-chat-sales"
column_mapping = {"customerId": "${data.customerId}", "question": "${data.question}"}

In [5]:
base_run = run_azure_flow(
    runtime, flow, run_name, data, column_mapping, pf_azure_client
)
pf_azure_client.stream(base_run)

Uploading salestestdata.jsonl (< 1 MB): 100%|██████████| 465/465 [00:00<00:00, 1.97kB/s]


Uploading contoso-chat (0.69 MBs): 100%|██████████| 690078/690078 [00:01<00:00, 381277.71it/s]


[2024-03-04 19:26:56,621][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ai.azure.com/projectflows/bulkrun/run/contoso-chat-sales_03_04_1926/details?wsid=/subscriptions/a195fdab-ef20-44e0-ae3a-48b2e1764e85/resourcegroups/contchat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'Running', continue streaming...)
2024-03-04 11:31:18 +0000     112 promptflow-runtime INFO     [contoso-chat-sales_03_04_1926] Receiving v2 bulk run request 62f6bc9f-ca42-4841-8525-94dc84ce3e6b: {"flow_id": "contoso-chat-sales_03_04_1926", "flow_run_id": "contoso-chat-sales_03_04_1926", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "9e3506e2-96eb-4ce0-8bb1-de71d877d222"}, "flow_dag_

In [6]:
details = pf_azure_client.get_details(base_run)
details.head(10)

,inputs.customerId,inputs.question,inputs.line_number,inputs.chat_history,outputs.answer,outputs.context
outputs.line_number,,,,,,
0,4,tell me about your hiking jackets,0,[],Hey Sarah Lee! 🌟 Let me tell you about our hik...,"[{'id': '17', 'title': 'RainGuard Hiking Jacke..."
1,1,Do you have any climbing gear?,1,[],"Hey John! 🧗‍♂️ Absolutely, we have some awesom...","[{'id': '9', 'title': 'SummitClimber Backpack'..."
2,3,Can you tell me about your selection of tents?,2,[],Hey Michael! 😄🏕️ Let me tell you about our awe...,"[{'id': '15', 'title': 'SkyView 2-Person Tent'..."
3,6,Do you have any hiking boots?,3,[],"Hey Emily! 👋 Absolutely, we've got some amazin...","[{'id': '4', 'title': 'TrekReady Hiking Boots'..."
4,2,What gear do you recommend for hiking?,4,[],"Hey Jane! 🌲 For hiking gear, I recommend the f...","[{'id': '10', 'title': 'TrailBlaze Hiking Pant..."


Running Evaluation

In [7]:
runtime = "automatic"
eval_flow = "multi_flow/"
data = "../data/salestestdata.jsonl"
run_name = "contoso-chat-sales-eval"

column_mapping = {
    # reference data
    "customerId": "${data.customerId}",
    "question": "${data.question}",
    "context": "${run.outputs.context}",
    # reference the run's output
    "answer": "${run.outputs.answer}",
}

In [8]:
eval_run = run_azure_eval_flow(
    runtime, eval_flow, run_name, data, column_mapping, base_run, pf_azure_client
)
pf_azure_client.stream(eval_run)

Uploading multi_flow (0.03 MBs): 100%|██████████| 34674/34674 [00:01<00:00, 25518.92it/s]


[2024-03-04 19:32:07,687][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ai.azure.com/projectflows/bulkrun/run/contoso-chat-sales-eval_03_04_1932/details?wsid=/subscriptions/a195fdab-ef20-44e0-ae3a-48b2e1764e85/resourcegroups/contchat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj
(Run status is 'NotStarted', continue streaming...)
2024-03-04 11:32:55 +0000      50 promptflow-runtime INFO     [contoso-chat-sales-eval_03_04_1932] Receiving v2 bulk run request ff529480-5559-476b-8fae-89ab187f2865: {"flow_id": "contoso-chat-sales-eval_03_04_1932", "flow_run_id": "contoso-chat-sales-eval_03_04_1932", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "99051f37-e1e7-48b7-bf89-320512583440"}, "flow_dag_file": "flow.dag.yaml"}, "connections": "**data_scrubbed**", "log_path": "https://stcontosoy46u2sqpv3h6g.blob.core.windows.net/8927edb2-b165-4c51-a19d-7201534af4ac-azureml/ExperimentRun/dcid.contoso-chat-sales-eval_03_04_1932/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b&sig=**data_sc

In [9]:
details = pf_azure_client.get_details(eval_run)
details.head(10)

,inputs.customerId,inputs.question,inputs.context,inputs.answer,inputs.line_number,inputs.chat_history,outputs.gpt_coherence,outputs.gpt_fluency,outputs.gpt_groundedness,outputs.gpt_relevance
outputs.line_number,,,,,,,,,,
0,4,tell me about your hiking jackets,"[{'id': '17', 'title': 'RainGuard Hiking Jacke...",Hey Sarah Lee! 🌟 Let me tell you about our hik...,0,[],5.0,5.0,5.0,5.0
1,1,Do you have any climbing gear?,"[{'id': '9', 'title': 'SummitClimber Backpack'...","Hey John! 🧗‍♂️ Absolutely, we have some awesom...",1,[],5.0,5.0,5.0,5.0
2,3,Can you tell me about your selection of tents?,"[{'id': '15', 'title': 'SkyView 2-Person Tent'...",Hey Michael! 😄🏕️ Let me tell you about our awe...,2,[],5.0,5.0,5.0,5.0
3,6,Do you have any hiking boots?,"[{'id': '4', 'title': 'TrekReady Hiking Boots'...","Hey Emily! 👋 Absolutely, we've got some amazin...",3,[],4.0,5.0,5.0,5.0
4,2,What gear do you recommend for hiking?,"[{'id': '10', 'title': 'TrailBlaze Hiking Pant...","Hey Jane! 🌲 For hiking gear, I recommend the f...",4,[],5.0,5.0,5.0,5.0


In [10]:
metrics = pf_azure_client.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

{
    "gpt_coherence": 4.8,
    "gpt_coherence_pass_rate(%)": 100.0,
    "gpt_groundedness": 5.0,
    "gpt_groundedness_pass_rate(%)": 100.0,
    "gpt_fluency": 5.0,
    "gpt_fluency_pass_rate(%)": 100.0,
    "gpt_relevance": 5.0,
    "gpt_relevance_pass_rate(%)": 100.0
}


In [11]:
pf_azure_client.visualize([base_run, eval_run])

Web View: https://ml.azure.com/prompts/flow/bulkrun/runs/outputs?wsid=/subscriptions/a195fdab-ef20-44e0-ae3a-48b2e1764e85/resourceGroups/contchat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj&runId=contoso-chat-sales_03_04_1926,contoso-chat-sales-eval_03_04_1932
